# Segment, localise and track


In [1]:
from macrohet import dataio, tile, notify
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
import dask.array as da
import glob
import zarr
import logging
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

# defining personal trained cellpose model to use
model_path = '/home/dayn/analysis/models/cellpose/PS0000/macrohet_seg'
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# ORRRR test the new cellpose model
# model = models.Cellpose(gpu=True, model_type='cyto3')

# Initialize the logging configuration
log_dir = "logs"  # Specify the directory where logs will be saved
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# Add a FileHandler to save logs to a file in the specified directory
log_file = os.path.join(log_dir, "assay_processing.log")
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s")
file_handler.setFormatter(formatter)
logging.getLogger().addHandler(file_handler)

# Define a function to log progress and potential errors
def log_progress(position, message):
    logging.info(f"Position {position}: {message}")


INFO:cellpose.models:>>>> loading model /home/dayn/analysis/models/cellpose/PS0000/macrohet_seg


Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
INFO:cellpose.models:>>>> model diam_labels =  188.746 (mean diameter of training ROIs)


### Define functions to tidy up main block of code

In [8]:
# define thresholds
segment_size_thresh = 5000
Mtb_load_thresh = 480

# define tracking scale factor
scale_factor = 1/5.04

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

# define tracker config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
# define tracker config fn to use
# config_fn = '/home/dayn/analysis/btrack/models/particle_config.json'

def segment(frame, model = model, channels = [0,0], diameter = 350, #250 #325
            min_size = 5000, model_type = 'pretrained'
           ):
    
    if model_type == 'pretrained':
        
        masks, flows, styles = model.eval(frame, # for default models
                                                 channels = channels, 
                                                 diameter = diameter, 
                                                 min_size = min_size, 
                                                 )
        
        
    else:

        masks, flows, styles = model.eval(frame, # for personal model
                                          channels = channels, 
                                          diameter = diameter, 
                                          min_size = min_size, 
                                          )
    return masks


def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects


def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks


def otsu_threshold_stack(images):
    """
    Function to characterise intra-Mφ Mtb load
    Computes Otsu's threshold value and returns a binary segmentation for
    each image in a time series of grayscale images.

    Parameters:
    -----------
    images : ndarray
        A 3D array of shape (n_images, height, width) containing a time series
        of grayscale images.

    Returns:
    --------
    ndarray
        A boolean array of shape (n_images, height, width) containing the
        binary segmentation for each image in the time series.
    """
    segmentations = np.zeros(images.shape, dtype=bool)
    for i, image in tqdm(enumerate(images), 
                         total=len(images), 
                         leave=False, 
                         desc='Otsu segmenting'):
        loaded_image = image.compute().compute()
        threshold = threshold_otsu(loaded_image)
        segmentations[i] = loaded_image > threshold
        
    return segmentations

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [3]:
base_dir = '/mnt/SYNO/macrohet_syno/data/ND0003/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,0,0.135256499,0,2024-02-16T17:15:25.597+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
1,0301K1F1P1R2,Ok,r03c01f01p01-ch2sk1fk1fl1.tiff,3,1,1,1,0,2,1,...,0,0.135256499,0,2024-02-16T17:15:25.813+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
2,0301K1F1P2R1,Ok,r03c01f01p02-ch1sk1fk1fl1.tiff,3,1,1,2,0,1,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.157+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
3,0301K1F1P2R2,Ok,r03c01f01p02-ch2sk1fk1fl1.tiff,3,1,1,2,0,2,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.39+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
4,0301K1F1P3R1,Ok,r03c01f01p03-ch1sk1fk1fl1.tiff,3,1,1,3,0,1,1,...,4E-06,0.135260597,0,2024-02-16T17:15:26.733+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349267,0612K154F9P1R2,Ok,r06c12f09p01-ch2sk154fk1fl1.tiff,6,12,9,1,153,2,1,...,0,0.135008901,275402.773,2024-02-19T21:59:46.84+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349268,0612K154F9P2R1,Ok,r06c12f09p02-ch1sk154fk1fl1.tiff,6,12,9,2,153,1,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.183+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349269,0612K154F9P2R2,Ok,r06c12f09p02-ch2sk154fk1fl1.tiff,6,12,9,2,153,2,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.4+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349270,0612K154F9P3R1,Ok,r06c12f09p03-ch1sk154fk1fl1.tiff,6,12,9,3,153,1,1,...,4E-06,0.135012895,275402.773,2024-02-19T21:59:47.743+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [4]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

# Segment, localise and track

In [5]:
mtb_channel = 0
gfp_channel = 1
manual_mtb_thresh_channel = 2
output_dirname = 'macrohet' #'cpv3_smaller_diam' #'cpv3'
# make output directory and subdirs 
os.makedirs(os.path.join(base_dir, f'labels/{output_dirname}/backup'), exist_ok = True)

# Just one position

In [7]:
row, column = 6, 3
acq_ID = (row, column)
log_progress(acq_ID, "Starting new acquisition")

# if info['Strain'] == 'UNI':
#     log_progress(acq_ID, "Skipping uninfected acquisition for now")
#     continue
# if acq_ID in already_processed_acq_IDs:
#     log_progress(acq_ID, "Skipping already processed")
#     continue
# if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_warea.h5')):
#     log_progress(acq_ID, "Skipping already processed")
#     continue

# process images using zarr
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_store = zarr.open(image_dir, mode='r')
images = zarr_store.images
# create a max projection
images = np.max(images, axis = 2)
# or just pick a z slice 
# images = zarr_store.images[:,:,0,...]

log_progress(acq_ID, "Images loaded and stacked")

# check if already segmented using m2 model
#     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')):
#         continue
#     else:
log_progress(acq_ID, "Starting segmentation")

#     if os.path.exists(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5')):
#         with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}_first_pass_seg_backup.h5'), 
#                                        'r', 
#                                        obj_type='obj_type_1'
#                                        ) as reader:
# #             writer.write_objects(objects)
#             # writer.write_tracks(tracks)
#             masks = reader.segmentation
#         log_progress(acq_ID, "Loaded previously calculated segmentation")
#     else:
# segment images from gfp channel only
masks = np.stack([segment(frame) 
                  for frame in tqdm(images[:,gfp_channel,...],  # segmenting the GFP channel 
                                    desc = 'Segmenting')])

log_progress(acq_ID, "Finished segmentation")

with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_mask_backup.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
#             writer.write_objects(objects)
        # writer.write_tracks(tracks)
        writer.write_segmentation(masks)
    
log_progress(acq_ID, "Saved out masks")  

log_progress(acq_ID, "Measuring Mtb area")       

# characterise Mtb growth using Otsu segmentation
# otsu_mtb = otsu_threshold(images[:,1,...]) # time consuming and non-deterministic when compared to hardcoded, could result in different thresholds for same image? 
# characterise Mtb growth using hardcoded threshold :S
manual_mtb_thresh = np.where(images[:,mtb_channel,...] >= Mtb_load_thresh, True, False)
log_progress(acq_ID, "Creating intensity image for localisation")  
# reshape intensity image to be gfp, rfp on last axis for regionprops
intensity_image = np.stack([images[:,0,...], 
                            images[:,1,...],  
#                                 otsu_mtb, 
                            manual_mtb_thresh], axis = -1)
log_progress(acq_ID, "Localising objects")  
# localise objects
objects = localise(masks, 
                   intensity_image, 
                   )
log_progress(acq_ID, "Filtering small objects")  
# filter out small objects
objects = [o for o in objects if o.properties['area'] > segment_size_thresh]

log_progress(acq_ID, "Adding infection labels to objects")  
# add label for infection
for obj in objects:
    obj.properties = ({"Infected": True} 
                        if obj.properties['mean_intensity'][manual_mtb_thresh_channel] > 0 # index 2 for manual mtb channel 
                        else {"Infected": False})
    obj.properties = ({"Mtb area px": obj.properties['mean_intensity'][manual_mtb_thresh_channel]*obj.properties['area']})

with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_objects_backup.h5'), 
                                           'w', 
                                           obj_type='obj_type_1'
                                           ) as writer:
                writer.write_objects(objects)
                # writer.write_tracks(tracks)

log_progress(acq_ID, "Beginning tracking")  
# track on upscaled config fn
tracks = track(objects, masks, config_fn, search_radius = 20)
log_progress(acq_ID, "Saving tracking")  
# save out 
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/backup/{row, column}_cpv3_tracks_backup.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
#             writer.write_objects(objects)
        writer.write_tracks(tracks)
        # writer.write_segmentation(masks)
# Log successful completion

with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/{output_dirname}/{row, column}_cpv3_tracks.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)
# Log successful completion
log_progress(acq_ID, "Processing completed successfully")

INFO:root:Position (6, 3): Starting new acquisition
INFO:root:Position (6, 3): Images loaded and stacked
INFO:root:Position (6, 3): Starting segmentation


Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 46.22 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 78.61 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 32.36 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 35.23 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 35.77 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 37.61 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 39.76 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 39.89 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 35.95 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 36.35 sec
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 36.40 sec
INFO:cellp

GLPK Integer Optimizer 5.0
16716 rows, 14033 columns, 19708 non-zeros
14033 integer variables, all of which are binary
Preprocessing...
8358 rows, 14033 columns, 19708 non-zeros
14033 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8358
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
8358 rows, 14033 columns, 19708 non-zeros
*     0: obj =   4.858774269e+04 inf =   0.000e+00 (3771)
Perturbing LP to avoid stalling [1861]...
Removing LP perturbation [3850]...
*  3850: obj =   2.318142536e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3850: mip =     not found yet >=              -inf        (1; 0)
+  3850: >>>>>   2.318142536e+04 >=   2.318142536e+04   0.0% (1; 0)
+  3850: mip =   2.318142536e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2024/07/01 08:58:54 PM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session
INFO:root:Position (6, 3): Saving tracking
[INFO][2024/07/01 08:58:54 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/backup/(6, 3)_cpv3_tracks_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/backup/(6, 3)_cpv3_tracks_backup.h5...
[INFO][2024/07/01 08:58:55 PM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/07/01 08:58:55 PM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/07/01 08:58:55 PM] Loading objects/obj_type_1 (36316, 5) (36316 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (36316, 5) (36316 filtered: None)
[INFO][2024/07/01 08:58:56 PM] Writing properties/obj_type_1/area (36316,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/area (36316,)
[INFO][2024/07/01 08:58:56 PM] Writing properties/obj_type_1/major_axis

# Iterate over all positions

In [9]:

# List of acq_IDs to process
valid_acq_IDs = [(5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9)]
processed_acq_IDs = set()

# while True:
for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout)):
    acq_ID = (row, column)
    
    # Skip if not in the valid_acq_IDs list
    if acq_ID not in valid_acq_IDs:
        continue
    
    # Skip if already processed
    if acq_ID in processed_acq_IDs:
        log_progress(acq_ID, "Already processed, skipping")
        continue
    
    try:
        log_progress(acq_ID, "Starting new acquisition")

        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')

        if os.path.exists(image_dir):
            images_subdir = os.path.join(image_dir, 'images')

            if os.path.exists(images_subdir) and os.path.isdir(images_subdir):
                file_count = len([f for f in os.listdir(images_subdir) if os.path.isfile(os.path.join(images_subdir, f))])

                if file_count == 925:
                    zarr_store = zarr.open(image_dir, mode='r')
                    images = zarr_store['images']
                    images = np.max(images, axis=2)

                    log_progress(acq_ID, "Images loaded and stacked")

                    log_progress(acq_ID, "Starting segmentation")
                    masks = np.stack([segment(frame, model_type='pretrained') for frame in tqdm(images[:, gfp_channel, ...], desc='Segmenting')])

                    log_progress(acq_ID, "Finished segmentation")

                    backup_path = os.path.join(base_dir, f'labels/{output_dirname}/{row},{column}_cpv3_mask_backup.h5')
                    with btrack.io.HDF5FileHandler(backup_path, 'w', obj_type='obj_type_1') as writer:
                        writer.write_segmentation(masks)

                    log_progress(acq_ID, "Saved out masks")

                    log_progress(acq_ID, "Measuring Mtb area")
                    manual_mtb_thresh = np.where(images[:, mtb_channel, ...] >= Mtb_load_thresh, True, False)
                    intensity_image = np.stack([images[:, 0, ...], images[:, 1, ...], manual_mtb_thresh], axis=-1)

                    log_progress(acq_ID, "Localising objects")
                    objects = localise(masks, intensity_image)

                    log_progress(acq_ID, "Filtering small objects")
                    objects = [o for o in objects if o.properties['area'] > segment_size_thresh]

                    log_progress(acq_ID, "Adding infection labels to objects")
                    for obj in objects:
                        obj.properties.update({
                            "Infected": obj.properties['mean_intensity'][manual_mtb_thresh_channel] > 0,
                            "Mtb area px": obj.properties['mean_intensity'][manual_mtb_thresh_channel] * obj.properties['area']
                        })

                    objects_backup_path = os.path.join(base_dir, f'labels/{output_dirname}/backup/{row},{column}_cpv3_objects_backup.h5')
                    with btrack.io.HDF5FileHandler(objects_backup_path, 'w', obj_type='obj_type_1') as writer:
                        writer.write_objects(objects)

                    log_progress(acq_ID, "Beginning tracking")
                    tracks = track(objects, masks, config_fn, search_radius=20)

                    tracks_backup_path = os.path.join(base_dir, f'labels/{output_dirname}/backup/{row},{column}_cpv3_tracks_backup.h5')
                    with btrack.io.HDF5FileHandler(tracks_backup_path, 'w', obj_type='obj_type_1') as writer:
                        writer.write_tracks(tracks)

                    final_path = os.path.join(base_dir, f'labels/{output_dirname}/{row},{column}.h5')
                    with btrack.io.HDF5FileHandler(final_path, 'w', obj_type='obj_type_1') as writer:
                        writer.write_tracks(tracks)
                        writer.write_segmentation(masks)

                    log_progress(acq_ID, "Processing completed successfully")
                    
                    # Mark as processed
                    processed_acq_IDs.add(acq_ID)

                else:
                    log_progress(acq_ID, f"Subdirectory {images_subdir} does not meet the file count requirement. It has {file_count} files.")
            else:
                log_progress(acq_ID, f"Subdirectory {images_subdir} does not exist or is not a directory.")
        else:
            log_progress(acq_ID, f"Directory {image_dir} does not exist.")
    except Exception as e:
        log_progress(acq_ID, f"Processing failed: {str(e)}")

logging.info("Processing completed")
notify.send_sms("Processing completed")

    # time.sleep(10 * 60)

Progress through positions:   0%|          | 0/42 [00:00<?, ?it/s]

INFO:root:Position (5, 3): Starting new acquisition
INFO:root:Position (5, 3): Images loaded and stacked
INFO:root:Position (5, 3): Starting segmentation


Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:root:Position (5, 3): Finished segmentation
[INFO][2024/07/10 06:11:27 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,3_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,3_cpv3_mask_backup.h5...
[INFO][2024/07/10 06:12:22 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,3_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,3_cpv3_mask_backup.h5
INFO:root:Position (5, 3): Saved out masks
INFO:root:Position (5, 3): Measuring Mtb area
INFO:root:Position (5, 3): Localising objects
[INFO][2024/07/10 06:12:45 PM] Localizing objects from segmentation...
INFO:btrack.io._localization:Localizing objects from segmentation...
[WARNING][2024/07/10 06:12:45 PM] Multichannel intensity image detected, using unweighted centroid.
progress: 100%|████████████████████████████████████████████| 154/154 [05:41<00:00,  2.22s/it]


GLPK Integer Optimizer 5.0
18708 rows, 15632 columns, 21910 non-zeros
15632 integer variables, all of which are binary
Preprocessing...
9354 rows, 15632 columns, 21910 non-zeros
15632 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 9354
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
9354 rows, 15632 columns, 21910 non-zeros
*     0: obj =   5.474701887e+04 inf =   0.000e+00 (4177)
Perturbing LP to avoid stalling [2072]...
Removing LP perturbation [4257]...
*  4257: obj =   2.614292300e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  4257: mip =     not found yet >=              -inf        (1; 0)
+  4257: >>>>>   2.614292300e+04 >=   2.614292300e+04   0.0% (1; 0)
+  4257: mip =   2.614292300e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

...

Segmenting:   0%|          | 0/154 [00:00<?, ?it/s]

INFO:root:Position (5, 9): Finished segmentation
[INFO][2024/07/11 10:24:25 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,9_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,9_cpv3_mask_backup.h5...
[INFO][2024/07/11 10:25:16 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,9_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/5,9_cpv3_mask_backup.h5
INFO:root:Position (5, 9): Saved out masks
INFO:root:Position (5, 9): Measuring Mtb area
INFO:root:Position (5, 9): Localising objects
[INFO][2024/07/11 10:25:38 AM] Localizing objects from segmentation...
INFO:btrack.io._localization:Localizing objects from segmentation...
[WARNING][2024/07/11 10:25:38 AM] Multichannel intensity image detected, using unweighted centroid.
progress: 100%|████████████████████████████████████████████| 154/154 [05:33<00:00,  2.17s/it]


GLPK Integer Optimizer 5.0
20236 rows, 17011 columns, 23904 non-zeros
17011 integer variables, all of which are binary
Preprocessing...
10118 rows, 17011 columns, 23904 non-zeros
17011 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 10118
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
10118 rows, 17011 columns, 23904 non-zeros
*     0: obj =   5.995526141e+04 inf =   0.000e+00 (4682)
Perturbing LP to avoid stalling [2208]...
Removing LP perturbation [4780]...
*  4780: obj =   2.842491850e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  4780: mip =     not found yet >=              -inf        (1; 0)
+  4780: >>>>>   2.842491850e+04 >=   2.842491850e+04   0.0% (1; 0)
+  4780: mip =   2.842491850e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

[INFO][2024/07/11 10:31:29 AM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session
[INFO][2024/07/11 10:31:30 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/backup/5,9_cpv3_tracks_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/macrohet/backup/5,9_cpv3_tracks_backup.h5...
[INFO][2024/07/11 10:31:30 AM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/07/11 10:31:30 AM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/07/11 10:31:30 AM] Loading objects/obj_type_1 (37182, 5) (37182 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (37182, 5) (37182 filtered: None)
[INFO][2024/07/11 10:31:31 AM] Writing properties/obj_type_1/area (37182,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/area (37182,)
[INFO][2024/07/11 10:31:31 AM] Writing properties/obj_type_1/major_axis_length (37182,)
INFO:btrack.io.hdf:Writi

Update sms failed to send
